<a href="https://colab.research.google.com/github/vkth7/AIFFEL-exploration/blob/main/exp-10/%5Bexp-10%5D%20%EB%8B%A8%EC%96%B4%20Level%EB%A1%9C%20%EB%B2%88%EC%97%AD%EA%B8%B0%20%EC%97%85%EA%B7%B8%EB%A0%88%EC%9D%B4%EB%93%9C%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 : 단어 Level로 번역기 업그레이드하기

In [ ]:
# !wget https://www.manythings.org/anki/fra-eng.zip

--2022-08-19 06:40:54--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6612164 (6.3M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   6.31M  7.24MB/s    in 0.9s    

2022-08-19 06:40:56 (7.24 MB/s) - ‘fra-eng.zip’ saved [6612164/6612164]

mv: target 'Notebooks/exp-10' is not a directory
/bin/bash: line 0: cd: too many arguments


In [ ]:
# !mv fra-eng.zip  /content/drive/MyDrive/Colab\ Notebooks/exp-10

In [ ]:
# !cd /content/drive/MyDrive/Colab\ Notebooks/exp-10 && unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install --upgrade tensorflow

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 7s (62.8 MB/s)
(Reading database ... 155676 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155654 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+c

실습에서 구현한 번역기는 글자 단위(Character-level)에서 구현된 번역기였습니다. 하지만 실제 번역기의 경우에는 글자 단위가 아니라 단어 단위(Word-level)에서 구현되는 것이 좀 더 보편적입니다.

동일한 데이터셋을 사용하면서 글자 단위와는 다른 전처리와 임베딩 층(Embedding layer)를 추가하여 단어 단위의 번역기를 완성시켜보겠습니다. 하지만, 단어 단위로 할 경우에는 단어의 개수가 글자 단위로 했을 경우와 비교하여 단어장의 크기(Vocabulary) 크기도 커지고, 학습 속도도 좀 더 느려집니다. 학습과 테스트 시의 원활한 진행을 위해서 **데이터에서 상위 33,000개의 샘플만 사용해주세요.**

**33000개 중 3000개는 테스트 데이터로 분리하여 모델을 학습한 후에 번역을 테스트 하는 용도로 사용합니다.**

**Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)**

---

글자 단위가 아닌 단어 단위의 번역기를 하기 위해서는 글자 단위에서는 신경쓰지 않았던 몇 가지 추가적인 전처리가 필요합니다.

1. **구두점(Punctuation)을 단어와 분리해주세요.**

일반적으로 영어권 언어의 경우에는 띄어쓰기 단위로 단어를 분리합니다. 토큰화(Tokenization)이라고도 불리는 이 작업은 어디서부터 어디까지가 하나의 단어인지를 구분하는 작업인데요. 그런데 띄어쓰기를 해주기 전에 구두점을 분리하는 작업이 필요할 때가 있습니다.

예를 들어서 'he is a good boy!'라는 문장이 있을 때, 이를 띄어쓰기 단위로 토큰화한다면

['he', 'is', 'a', 'good', 'boy!']가 됩니다. 그런데 실제로 !는 boy와 붙어있는 한 단어가 아니므로

좀 더 올바른 전처리는 ['he', 'is', 'a', 'good', 'boy', '!']가 맞습니다.

!나 ? 또는 온점과 같은 특수문자들을 구두점(punctuation)이라고 부릅니다.

이들을 토큰화하기 전에 단어와 미리 분리시켜주세요!

분리 전 : he is a Good boy!

분리 후 : he is a Good boy !

2. **소문자로 바꿔주세요.**

기계가 보기에는 스펠링이 같더라도 대문자로 된 단어와 소문자로 된 단어는 서로 다른 단어입니다. 예를 들어 'Good'과 'good'은 기계가 보기에는 다른 단어입니다. 그래서 모든 문장에 대해서 전부 영어로 바꿔주는 작업을 하겠습니다.

변환 전 : he is a Good boy !

변환 후 : he is a good boy !

3. **띄어쓰기 단위로 토큰를 수행하세요.**

띄어쓰기 단위로 토큰화를 수행해서 단어를 분리하는 작업을 해주세요. 기계는 이렇게 분리된 토큰들을 각각 하나의 단어로 인식할 수 있게 됩니다.

토큰화 전 : 'he is a good boy !'

토큰화 후 : ['he', 'is', 'a', 'good', 'boy', '!']

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [2]:
file_path = '/content/drive/MyDrive/Colab Notebooks/exp-10/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 194513


,eng,fra,cc
137148,You have a good chance of winning.,Vous avez une bonne chance de gagner.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
35583,I have a pretty dog.,J'ai un beau chien.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
175374,The police are looking into his disappearance.,La police est en train d'enquêter sur sa dispa...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
76639,Don't forget to add me in.,N'oublie pas de m'y ajouter.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
154884,Will the work be finished by tomorrow?,L’œuvre sera-t-elle achevée d'ici demain ?,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [3]:
lines = lines[['eng', 'fra']][:33000] # 3만3천개 샘플 사용
lines.sample(5)

,eng,fra
8488,That's a fake.,C'est un faux.
8337,She is French.,Elle est française.
23408,I also went there.,J'y suis également allée.
24093,I pruned the tree.,J'ai élagué l'arbre.
32143,Tom became popular.,Tom est devenu populaire.


In [4]:
import re
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. 다시 양쪽 공백을 지웁니다
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = sentence.strip() # 4
    return sentence

In [5]:
lines.fra = lines.fra.apply(lambda x : preprocess_sentence(x))
lines.eng = lines.eng.apply(lambda x : preprocess_sentence(x))

In [6]:
# 시작 토큰과 종료 토큰 추가
sos_token = '\t'
eos_token = '\n'
lines.fra = lines.fra.apply(lambda x : '\t '+ x + ' \n')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 33000


,eng,fra
3461,i'm through .,\t j'en ai terminé . \n
8761,tom is ok now .,\t thomas est ok maintenant . \n
24037,i need to do that .,\t je dois faire ça . \n
6388,will tom die ?,\t tom va-t-il mourir ? \n
11851,they were dead .,\t ils étaient morts . \n


In [7]:
lines.fra = lines.fra.str.split(' ')
lines.eng = lines.eng.str.split(' ')
lines.head()

,eng,fra
0,"[go, .]","[\t, va, !, \n]"
1,"[go, .]","[\t, marche, ., \n]"
2,"[go, .]","[\t, bouge, !, \n]"
3,"[hi, .]","[\t, salut, !, \n]"
4,"[hi, .]","[\t, salut, ., \n]"


In [8]:
eng_tokenizer = Tokenizer(char_level=True)   # 단어 단위로 Tokenizer를 생성
eng_tokenizer.fit_on_texts(lines.eng)               # 30000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

[[26, 1], [26, 1], [26, 1]]

In [9]:
fra_tokenizer = Tokenizer(char_level=True)   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:3]

[[1, 63, 7, 2], [1, 370, 3, 2], [1, 728, 7, 2]]

In [10]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4809
프랑스어 단어장의 크기 : 10879


In [11]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

In [12]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4809
프랑스어 단어장의 크기 : 10879
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 15


In [13]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [14]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 63, 7], [1, 370, 3], [1, 728, 7]]
[[63, 7, 2], [370, 3, 2], [728, 7, 2]]


In [15]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 15)
프랑스어 출력데이터의 크기(shape) : (33000, 15)


In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 학습데이터의 크기(shape) : (33000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (33000, 15)
프랑스어 학습 출력데이터의 크기(shape) : (33000, 15)


In [17]:
embedding_dim = 64
hidden_units = 64

In [18]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(units=hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [19]:
# # 입력 텐서 생성.
# encoder_inputs = Input(shape=(None,))
# # hidden size가 256인 인코더의 LSTM 셀 생성
# encoder_lstm = LSTM(units = 256, return_state = True)
# # 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
# encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# # hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
# encoder_states = [state_h, state_c]

In [20]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(fra_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(units=hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
# # 입력 텐서 생성.
# decoder_inputs = Input(shape=(None, fra_vocab_size))
# # hidden size가 256인 인코더의 LSTM 셀 생성
# decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# # decoder_outputs는 모든 time step의 hidden state
# decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [22]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [23]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 64)     307776      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 64)     696256      ['input_2[0][0]']                
                                                                                              

In [24]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
235/235 [==============================] - 75s 255ms/step - loss: 8.8600 - accuracy: 0.6232 - val_loss: 8.4444 - val_accuracy: 0.5836
Epoch 2/50
235/235 [==============================] - 57s 241ms/step - loss: 8.0453 - accuracy: 0.6259 - val_loss: 7.7037 - val_accuracy: 0.5836
Epoch 3/50
235/235 [==============================] - 57s 241ms/step - loss: 7.2990 - accuracy: 0.6259 - val_loss: 6.9926 - val_accuracy: 0.5836
Epoch 4/50
235/235 [==============================] - 57s 241ms/step - loss: 6.5843 - accuracy: 0.6259 - val_loss: 6.3195 - val_accuracy: 0.5836
Epoch 5/50
235/235 [==============================] - 57s 241ms/step - loss: 5.9094 - accuracy: 0.6259 - val_loss: 5.6928 - val_accuracy: 0.5836
Epoch 6/50
235/235 [==============================] - 57s 241ms/step - loss: 5.2759 - accuracy: 0.6259 - val_loss: 5.1077 - val_accuracy: 0.5836
Epoch 7/50
235/235 [==============================] - 57s 241ms/step - loss: 4.6795 - accuracy: 0.6259 - val_loss: 4.5604 - val_ac

In [25]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 64)          307776    
                                                                 
 masking (Masking)           (None, None, 64)          0         
                                                                 
 lstm (LSTM)                 [(None, 64),              33024     
                              (None, 64),                        
                              (None, 64)]                        
                                                                 
Total params: 340,800
Trainable params: 340,800
Non-trainable params: 0
_________________________________________________________________


In [26]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(hidden_units,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [27]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, None, 64)     696256      ['input_2[0][0]']                
                                                                                                  
 input_3 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 64)]         0           []                               
                                                                                            

In [28]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [30]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fra2idx['\t']

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' ' + sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [31]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

ValueError: ignored